In [12]:
import numpy as np
from numpy import ndarray
from abc import ABC, abstractmethod

In [13]:
def assert_same_shape(array: ndarray, array_grad: ndarray):
    assert array.shape == array_grad.shape, \
    f'''
    Two ndarrays should have the same shape;
    instead, first ndarray's shape is {array.shape}
    and second ndarray's shape is {array_grad.shape}
    '''
    return None

<h1> <code>Operation</code> and <code>ParamOperation</code></h1>

In [14]:
# abstract class for any Operation
class Operation(ABC):
    def __init__(self):
        pass
    
    def forward(self, input_: ndarray):
        '''
        Stores input in the self._input instance variable 
        Calls the self._output() function
        '''
        self.input_ = input_
        
        self.output = self._output()
        
        return self.output
    
    
    def backward(self, output_grad: ndarray):
        '''
        Calls the self._input_grad() function.
        Checks that the appropriate shapes match.
        '''
        assert_same_shape(self.output, output_grad)
        
        self.input_grad = self._input_grad(output_grad)
        
        assert_same_shape(self.input_, self._input_grad)
        
        return self.input_grad
    
    @abstractmethod
    def _output(self): 
        '''
        The output method must be defined for each Operation
        '''
        pass
    
    
    @abstractmethod
    def _input_grad(self, output_grad: ndarray) -> ndarray:
        '''
        The _input_grad method must be defined for each Operation
        '''
        pass
    

In [5]:
# Another abstract class for "parameter" operations
class ParamOperation(Operation):
    
    def __init__(self, param: ndarray):
        super().__init__()
        self.param = param
        
    
    def backward(self, output_grad: ndarray):
        '''
        Calls the self._input_grad and self._param_grad.
        Checks appropriate shapes
        '''
        assert_same_shape(self.output, output_grad)
        
        self.input_grad = self._input_grad(output_grad)
        self.param_grad = self._param_grad(output_grad)
        
        assert_same_shape(self.input_, self.input_grad)
        assert_same_shape(self.param, self.param_grad)
    
    
    @abstractmethod
    def _param_grad(self, output_grad: ndarray):
        '''
        Every subclass of ParamOperation must implement _param_grad
        '''
        pass

# Specific Operations

## 1. weight multiply

In [15]:
class WeightMultiply(ParamOperation):
    '''
    Weight multiplication operation for a neural network
    '''
    def __init__(self, W: ndarray):
        '''Initialize Operation with self.param = W'''
        super().__init__(W)
        
    
    def _output(self) -> ndarray:
        '''Compute the output'''
        return np.dot(self.input_, self.param)
    
    
    def _input_grad(self, output_grad: ndarray) -> ndarray: 
        '''Compute the gradient'''
        return np.dot(output_grad, np.transpose(self.param, (1, 0)))
    
    
    def _param_grad(self, output_grad: ndarray) -> ndarray:
        '''Compute the parameter gradient'''
        return np.dot(np.transpose(self.input_, (1, 0)), output_grad) 
        

## 2. bias add

In [16]:
class BiasAdd(ParamOperation):
    '''
    Compute bias addition
    '''
    
    def __init__(self, B: ndarray):
        
        assert B.shape[0] == 1
        
        super().__init__(B)
        
    
    def _output(self) -> ndarray:
        '''
        Compute output
        '''
        return self.input_ + self.param
    
    
    def _input_grad(self, output_grad: ndarray):
        '''
        Compute input gradient
        '''
        return np.ones_like(self.input_) * output_grad
    
    
    def _param_grad(self, ouput_grad: ndarray):
        '''Compute the param grad'''
        param_grad = np.ones_like(self.param) * output_grad
        return np.sum(param_grad, axis=0).reshape(1, param_grad.shape[1])

## 3. sigmoid

In [37]:
class Sigmoid(Operation):
    '''
    Sigmoid activaiton function
    '''
    def __init__(self):
        super().__init__()
        
    def _output(self):
        return 1.0 /(1.0 + np.exp(-1.0 * self.input_))
    
    def _input_grad(self, output_grad: ndarray): 
        sigmoid_backward = self.output * (1 - self.output) 
        input_grad = sigmoid_backward * output_grad
        return input_grad
        

## 4. linear

In [38]:
class Linear(Operation):
    '''
    Identity activation function
    '''
    
    def __init__(self):
        super().__init__()
        
    def _output(self):
        return self.input_
    
    def _input_grad(self, output_grad: ndarray): 
        return output_grad

<h1><code>Layer</code> and <code>Dense</code></h1>

In [39]:
class Layer(ABC):
    '''
    A layer of neurons in a neural network
    '''
    
    def __init__(self, neurons: int):
        '''
        The number of neurons roughly corresponds to the breadth of the layer
        '''
        self.neurons = neurons
        self.first = True
        self.params: list[ndarray] = []
        self.param_grads: list[ndarray] = []
        self.operations: list[Operation] = []
    
    
    @abstractmethod
    def _setup_layer(self, num_in: int):
        '''The _setup_layer function must be implemented for each layer'''
        pass
    
    def forward(self, input_: ndarray):
        '''
        Passes input forward through a series of operations
        '''
        
        if self.first:
            self._setup_layer(input_)
            self.first = False
        
        self.input_ = input_
        
        for operation in self.operations:
            input_ = operation.forward(input_)
            
        self.output = input_
        
        return self.output
    
    
    def backward(self, output_grad: ndarray):
        '''
        Passes output_grad backward through a series of operations
        Checks appropriate shapes
        '''
        assert_same_shape(self.output, output_grad)
        
        for operation in reversed(self.operations):
            output_grad = operation.backward(output_grad)
            
        input_grad = output_grad
        
        self._param_grads()
        
        return input_grad
    
    def _param_grads(self):
        '''
        Extracts the _param_grads from a layer's operations
        '''
        self.param_grads = []
        for operation in self.operations:
            if issubclass(operation.__class__, ParamOperation):
                self.param_grads.append(operation.param_grad)
        
    
    def _params(self):
        '''Extract params'''
        self.params = []
        for operation in self.operations:
            if issubclass(operation.__class__, ParamOperation):
                self.params.append(operation.param)

In [40]:
class Dense(Layer):
    '''A fully connected layer which inherits from Layer'''
    def __init__(self, neurons: int, activation: Operation = Sigmoid()):
        super().__init__(neurons)
        self.activation = activation
    
    def _setup_layer(self, input_: ndarray):
        '''Operations'''
        
        if self.seed:
            np.random.seed(self.seed)
            
        self.params = []
        
        # weights
        self.params.append(np.random.randn(input_.shape[1], self.neurons))
        
        # bias 
        self.params.append(np.random.randn(1, self.neurons))
        
        self.operations = [WeightMultiply(self.params[0]),
                           BiasAdd(self.params[1]),
                           self.activation
                          ]
        return None
        

In [41]:
class Loss(ABC):
    """The loss of a neural network"""
    
    def __init__(self):
        pass
    
    def forward(self, prediction: ndarray, target: ndarray):
        '''Computes the actual loss value'''
        assert_same_shape(prediction, target)
        
        self.prediction = prediction
        self.target = target
        
        loss_value = self._output()
        
        return loss_value
    
    
    def backward(self): 
        '''Computes gradient of the loss value with respect to the input to the loss function'''
        self.input_grad = self._input_grad()
        
        assert_same_shape(self.prediction, self.input_grad)
        
        return self.input_grad
    
    
    @abstractmethod
    def _output(self):
        pass
    
    
    @abstractmethod
    def _input_grad(self):
        pass
        

In [ ]:
class MeanSquaredError(Loss):
    def __init__(self):
        super().__init__()
        
    def _output(self):
        loss = (
            np.sum(np.power(self.prediction - self.target, 2)) / self.prediction.shape[0]
        )
        
        return loss
    
    def _input_grad(self):
        ret